全局变量：
        #生长：
        
        存储能量比率：energystorage=100％
        能量消耗速度：speed=1时，energyrate=/time   speed=2时，energyrate=/time  呈平方递推  （需要一个基准物种）
        饥饿比率：hungrynum=30％

生长：
        给生物自身设置一个时间表，记录其经历的刷新次数，通过刷新次数决定，每次从食物中扣除多少用以自身生长。
        大概24刷新的时间进行一次捕食
        100次刷新成长一岁
        正常年纪的30％为成年期
        前百分之30：energy=-500/(9*(t^2))*(x^2)+100/(3*t)*x+1
        后百分之70：energy=-500/(49*(t^2))*(x^2)+300/(49*t)*x+249/49
        假设成年体重是幼年体中的六倍
        幼年能量记为authenweight
        各年龄能量呈正态分布，方差为0.25authenweight
        
捕食：
        捕食对象的能量含量：preyenergy

运动：
        饥饿状态下的运动是先找到运动方向，再朝着这个方向一直运动
搜索：
        搜索所有该动物的列表，看是否有位置在感知范围内的动物
        无，返回0，0
        有，返回1，最近的该物种
记忆：
        确定是否是水源，在确定是否存在于记忆之中，若是水源且不存在记忆之中，就把该位置储存到memory_data里
捕猎：
        按照给定速度
        确定与对象的距离是否能在一回合内接近
        刚开始消耗耐力值进行高速奔跑
        耐力值耗尽后以正常速度奔跑
        捕食成功返回1，未成功返回0
耐力值系统：
        每time回复5，高速消耗25
        耐力值归零后，耐力值判定为0，动物不能继续奔跑
        等到耐力值回到最大值后，耐力值判定为1，动物可以选择继续奔跑
逃跑过程：
        按照给定速度
        确定与对象的距离是否能在一回合内接近
        刚开始消耗耐力值进行高速奔跑
        耐力值耗尽后以正常速度奔跑
        被捕食成功返回1，未成功返回0
渴系统：
        渴度为100
        每time消耗1
觅水行为：
        以最近距离前往记忆中的水位
        先确定去哪个水位，并返回该水位
        在行动
求偶行为：
        首先搜索对象
        对象性别必须为异性
        沿用饥饿的方向函数
        找到对象之后，二者向对方的位置前进
        到达同一位置则返回1和最远位置坐标，否则返回0和最远位置坐标

In [197]:
import numpy as np
import math

In [44]:
class biology:
    def __init__(self,speed,visibility,age,ageaverage,authenweight,energyweight,energystorage,setting,food,enermy):#分别代表生物的运动能力，感知范围，年龄,寿命,幼年体重,
        #，初始体重，初始储存能量,初始位置，食物,天敌
        self.no_is=1  #存活判定
        self.speed=speed
        self.visibility=visibility
        self.age=float(age)
        self.ageaverage=ageaverage
        self.authenweight=authenweight
        self.energyweight=energyweight
        self.energystorage=energystorage
        self.setting=setting
        self.food=food
        self.enermy=enermy
        self.memory_data=[]
        self.bear=100
        self.thirs=100
        self.hungry=1#是否饥饿
        self.seekis=0#是否开始寻找
        
    def timemaker(self):
        self.time=self.age*100
        self.time=self.time+1
        self.age=self.time/100
        

    def age_energy_maker(self):#生长过程
        if self.age<=self.ageaverage*0.3:
            weightaverage=(-500/(9*(self.ageaverage**2))*(self.age**2)+100/(3*self.ageaverage)*self.age+1)*self.authenweight
        elif (self.age>self.ageaverage*0.3) & (self.age<self.ageaverage):
            weightaverage=(-500/(49*(self.ageaverage**2))*(self.age**2)+300/(49*self.ageaverage)*self.age+249/49)*self.authenweight
        else:
            self.no_is=0
            weightaverage=self.authenweight
        result = np.random.normal(weightaverage, 0.25*self.authenweight, 1)
        a=self.energyweight
        self.energyweight=result[0]
        self.energystorage=self.energystorage-(self.energyweight-a)

    def walk(self):#饱食状态下的运动过程
        b=[]
        for i in range(self.setting[0]-self.speed,self.setting[0]+self.speed+1):
            for j in range(self.setting[1]-self.speed,self.setting[1]+self.speed+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.speed:
                    b.append([i,j])
        c=np.random.randint(0,len(b))
        self.setting=b[c]

    def seekfor(self):#刚开始饥饿运动时，要先确定寻找的方向
        pass

    def seek(self):#饥饿状态下的觅食运动
        if (self.hungry==1) & (self.seekis==0):
            self.x,self.y=self.seekfor()
            seekis=1
        self.setting[0]+=self.x
        self.setting[1]+=self.y

    def see(self,cate):#搜索函数
        b=[]
        for i in range(self.setting[0]-self.visibility,self.setting[0]+self.visibility+1):
            for j in range(self.setting[1]-self.visibility,self.setting[1]+self.visibility+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.visibility:
                    b.append([i,j])
        c=[]
        for m in cate:
            for n in b:
                if m.setting==n:
                    c.append(m)
        if c==[]:
            return(0,0)
        else:
            d=c[0]
            for i in c:
                x=((i.setting[0]-self.setting[0])**2+(i.setting[1]-self.setting[1])**2)**0.5
                y=((d.setting[0]-self.setting[0])**2+(d.setting[1]-self.setting[1])**2)**0.5
                if x<=y:
                    d=i
            return(1,d)

    def memory(self,water):
        for i in water:
            a=1
            for j in self.memory_data:
                if i==j:
                    a=0
            if (a==1)&(self.setting==i):
                self.memory_data.append(i)

    def bear_get(self):#耐力值获取系统
        self.bear+=5
        if self.bear>=100:
            self.bear=100
            self.quick_is=1

    def bear_dec(self):#耐力值消耗系统
        self.bear-=25
        if self.bear<=0:
            self.bear=0
            self.quick_is=0

    def predation(self,food,speed_s,speed_i):#捕食过程
        a=((food.setting[0]-self.setting[0])**2+(food.setting[1]-self.setting[1])**2)**0.5
        if a<=(speed_s-speed_i):
            self.setting=food.setting
            return 1
        else:
            for i in range(self.setting[0]-speed_s,self.setting[0]+speed_s+1):
                for j in range(self.setting[1]-speed_s,self.setting[1]+speed_s+1):
                    c=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                    if c<=speed_s:
                        b.append([i,j])
            n=b[0]
            for m in b:
                d=((m[0]-food.setting[0])**2+(m[1]-food.setting[1])**2)**(0.5)
                e=((n[0]-food.setting[0])**2+(n[1]-food.setting[1])**2)**(0.5)
                if d<=e:
                    n=m
            self.setting=n
            return 0

    def escape(self,enermy,speed_s,speed_i):#逃跑过程
        a=((enermy.setting[0]-self.setting[0])**2+(enermy.setting[1]-self.setting[1])**2)**0.5
        if a<=(speed_i-speed_s):
            self.no_is=0
            return 1
        else:
            for i in range(self.setting[0]-speed_s,self.setting[0]+speed_s+1):
                for j in range(self.setting[1]-speed_s,self.setting[1]+speed_s+1):
                    c=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                    if c<=speed_s:
                        b.append([i,j])
            n=b[0]
            for m in b:
                d=((m[0]-enermy.setting[0])**2+(m[1]-enermy.setting[1])**2)**(0.5)
                e=((n[0]-enermy.setting[0])**2+(n[1]-enermy.setting[1])**2)**(0.5)
                if d>=e:
                    n=m
            self.setting=n
            return 0

    
    def thirsty(self):#渴觉系统
        self.thirs-=2
    
    def seekforwater(self):#觅水行为,确定方向
        a=self.memory_data[0]
        for i in self.memory_data:
            m=((a[0]-self.setting[0])**2+(a[1]-self.setting[1])**2)**(0.5)
            n=((i[0]-self.setting[0])**2+(i[1]-self.setting[1])**2)**(0.5)
            if n<=m:
                a=i
        return a

    def runforwater(self,target):#开始觅水
        b=[]
        for i in range(self.setting[0]-self.speed,self.setting[0]+self.speed+1):
            for j in range(self.setting[1]-self.speed,self.setting[1]+self.speed+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.speed:
                    b.append([i,j])
        c=b[0]
        for i in b:
            m=((c[0]-target[0])**2+(c[1]-target[1])**2)**(0.5)
            n=((i[0]-target[0])**2+(i[1]-target[1])**2)**(0.5)
            if n<=m:
                c=i
        self.setting=c

    def courtship(self,target):
        x=((self.setting[0]-target.setting[0])**2+(self.setting[1]-target.setting[1])**2)**0.5
        b=[]
        for i in range(self.setting[0]-self.speed,self.setting[0]+self.speed+1):
            for j in range(self.setting[1]-self.speed,self.setting[1]+self.speed+1):
                a=((i-self.setting[0])**2+(j-self.setting[1])**2)**(0.5)
                if a<=self.speed:
                    b.append([i,j])
        c=b[0]
        for i in b:
            m=((c[0]-target[0])**2+(c[1]-target[1])**2)**(0.5)
            n=((i[0]-target[0])**2+(i[1]-target[1])**2)**(0.5)
            if n<=m:
                c=i
        if x<=self.speed+target.speed:
            return 1,c
        else:
            return 0,c

    
        

            
        
    def growth(self):#生物的生长过程
        pass



In [46]:
a=biology(1,2,1,100,1,1,1,[6,6],1)
food=[]
x=[]
for i in range (100):
    food.append(biology(1,2,1,100,1,1,1,[i,6],0))
for i in range (100):
    x.append(biology(1,2,1,100,1,1,1,[i,5],food))
print(x[0].food[1].setting)
c,d=x[6].see(x[6].food)
print(d.setting)
data=[[6,4],[6,5],[6,6]]
x[6].memory(data)

[1, 6]
[[4, 5], [5, 4], [5, 5], [5, 6], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [7, 4], [7, 5], [7, 6], [8, 5]]
[6, 6]
[[6, 5]]


In [230]:

a.timemaker()
a.age_energy_maker()
a.seek()
print(a.energyweight)
print(a.energystorage)
print(a.age)
print(a.setting)

1.0


TypeError: cannot unpack non-iterable NoneType object

In [129]:
for i in range (1,4):
    print(i)

1
2
3


In [152]:
print(len([[1,23],[1,2]]))

2


SyntaxError: invalid syntax (394029582.py, line 1)